In [22]:
# Hyperparameter Sensitivity Analysis
# This notebook explores the sensitivity of the model's performance to various hyperparameters using a grid search approach.

print(os.getcwd())
#os.chdir("projects/ocr")
import itertools
import numpy as np
import pandas as pd
from datasets import load_from_disk
from transformers import Qwen2VLForConditionalGeneration, Qwen2_5_VLForConditionalGeneration, AutoProcessor
import torch
from qwen_vl_utils import process_vision_info
from PIL import Image
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import torch
import time
import os

from src.train_test import  run_inference_and_calculate_cer
from src.qwen_finetune import train_and_validate


/home/alex/projects/ocr


In [24]:
import itertools
import os
import torch
import time
from src.train_test import  run_inference_and_calculate_cer
from src.qwen_finetune import train_and_validate
from datasets import load_dataset


In [25]:

# Define hyperparameter grid
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [4, 8]
accumulation_steps = [1, 2]
dataset_ranges = [(0, 55), (0, 100)]
image_factors = [28, 32]
eval_steps_list = [0, 0]
max_steps_list = [50, 100]

# Create combinations of hyperparameters
param_grid = list(itertools.product(learning_rates, batch_sizes, accumulation_steps, dataset_ranges, image_factors, eval_steps_list, max_steps_list))


In [28]:
param_grid


[(1e-05, 4, 1, (0, 55), 28, 1000, 5000),
 (1e-05, 4, 1, (0, 55), 28, 1000, 10000),
 (1e-05, 4, 1, (0, 55), 28, 2000, 5000),
 (1e-05, 4, 1, (0, 55), 28, 2000, 10000),
 (1e-05, 4, 1, (0, 55), 32, 1000, 5000),
 (1e-05, 4, 1, (0, 55), 32, 1000, 10000),
 (1e-05, 4, 1, (0, 55), 32, 2000, 5000),
 (1e-05, 4, 1, (0, 55), 32, 2000, 10000),
 (1e-05, 4, 1, (0, 100), 28, 1000, 5000),
 (1e-05, 4, 1, (0, 100), 28, 1000, 10000),
 (1e-05, 4, 1, (0, 100), 28, 2000, 5000),
 (1e-05, 4, 1, (0, 100), 28, 2000, 10000),
 (1e-05, 4, 1, (0, 100), 32, 1000, 5000),
 (1e-05, 4, 1, (0, 100), 32, 1000, 10000),
 (1e-05, 4, 1, (0, 100), 32, 2000, 5000),
 (1e-05, 4, 1, (0, 100), 32, 2000, 10000),
 (1e-05, 4, 2, (0, 55), 28, 1000, 5000),
 (1e-05, 4, 2, (0, 55), 28, 1000, 10000),
 (1e-05, 4, 2, (0, 55), 28, 2000, 5000),
 (1e-05, 4, 2, (0, 55), 28, 2000, 10000),
 (1e-05, 4, 2, (0, 55), 32, 1000, 5000),
 (1e-05, 4, 2, (0, 55), 32, 1000, 10000),
 (1e-05, 4, 2, (0, 55), 32, 2000, 5000),
 (1e-05, 4, 2, (0, 55), 32, 2000, 1000

In [29]:

# Select the first set of parameters
params = param_grid[0]
lr, batch_size, acc_steps, (train_start, train_end), img_factor, eval_steps, max_steps = params


In [31]:

# Set up output directory
output_dir = f'results/predictions/models/lr_{lr}_bs_{batch_size}_acc_{acc_steps}_range_{train_start}_{train_end}_img_{img_factor}_eval_{eval_steps}_max_{max_steps}'
output_dir

'results/predictions/models/lr_1e-05_bs_4_acc_1_range_0_55_img_28_eval_1000_max_5000'

In [32]:

os.makedirs(output_dir, exist_ok=True)


In [33]:

# Load dataset
dataset_p40 = load_from_disk("data/processed/dataset_p40")


In [34]:

# Start timing
start_time = time.time()

# Train model
try:
    train_and_validate(
        model_name='Qwen/Qwen2-VL-2B-Instruct',
        output_dir=output_dir,
        dataset_name='culturalheritagenus/Gongguan-OCR-p40',
        image_column='image',
        text_column='text',
        device='cuda' if torch.cuda.is_available() else 'cpu',
        min_pixel=256,
        max_pixel=384,
        image_factor=img_factor,
        num_accumulation_steps=acc_steps,
        eval_steps=eval_steps,
        max_steps=max_steps,
        train_select_start=train_start,
        train_select_end=train_end,
        val_select_start=0,
        val_select_end=14,
        train_batch_size=batch_size,
        val_batch_size=1
    )
except Exception as e:
    print(f"Error during training: {e}")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.82it/s]
Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2-VL-2B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error during training: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.


In [35]:
def run_inference_and_calculate_cer(model_path, message, dataset):
    # Load the model and processor
    model = Qwen2VLForConditionalGeneration.from_pretrained(model_path, device_map="auto").half()
    processor = AutoProcessor.from_pretrained(model_path)

    messages_template = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": None,  # Placeholder for image, to be updated
                },
                {"type": "text", "text": f"{message}"},
            ],
        }
    ]

    results = []
    # Run inference on the dataset
    for item in dataset:
        prediction = process_image(item["image"], model, message_template)
        cer = compute_CER(item["text"], prediction, normalize=True)
        results.append({
            "ground_truth": item["text"],
            "num_lines": item["num_lines"],
            "prediction": prediction,
            "identifier": item["identifier"],
            "CER": cer
        })

    # Create DataFrame from results
    df = pd.DataFrame(results)

    return df


In [37]:

# Evaluate model

message = "Convert this image to text"

# Evaluate model
df_results = run_inference_and_calculate_cer(output_dir, message, dataset_p40)
    



OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory results/predictions/models/lr_1e-05_bs_4_acc_1_range_0_55_img_28_eval_1000_max_5000.

In [ ]:

try:
    df_results = run_inference_and_calculate_cer(output_dir, messages_template, dataset_p40)
    # Calculate metrics
    mean_cer = df_results['CER'].mean()
    median_cer = df_results['CER'].median()
    memory_usage = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    speed = max_steps / (time.time() - start_time)

    print(f"Mean CER: {mean_cer}")
    print(f"Median CER: {median_cer}")
    print(f"Memory Usage: {memory_usage}")
    print(f"Speed: {speed} steps/sec")
except Exception as e:
    print(f"Error during evaluation: {e}")

In [12]:

# Function to train and evaluate a model with given hyperparameters
def train_and_evaluate(params):
    lr, batch_size, acc_steps, (train_start, train_end), img_factor, eval_steps, max_steps = params
    output_dir = f'models/lr_{lr}_bs_{batch_size}_acc_{acc_steps}_range_{train_start}_{train_end}_img_{img_factor}_eval_{eval_steps}_max_{max_steps}'
    os.makedirs(output_dir, exist_ok=True)
    
    # Train model
    train_and_validate(
        model_name='Qwen/Qwen2-VL-2B-Instruct',
        output_dir=output_dir,
        dataset_name='culturalheritagenus/Gongguan-OCR-p40',
        image_column='image',
        text_column='text',
        device='cuda' if torch.cuda.is_available() else 'cpu',
        min_pixel=256,
        max_pixel=384,
        image_factor=img_factor,
        num_accumulation_steps=acc_steps,
        eval_steps=eval_steps,
        max_steps=max_steps,
        train_select_start=train_start,
        train_select_end=train_end,
        val_select_start=0,
        val_select_end=14,
        train_batch_size=batch_size,
        val_batch_size=1
    )
    
    messages_template = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": None,  # Placeholder for image, to be updated
                },
                {"type": "text", "text": "Convert this image to text"},
                #{"type": "text", "text": "Convert this image to Chinese text and carefully consider when the image has multiple columns of text"},
            ],
        }
    ]

    # Evaluate model
    df_results = run_inference_and_calculate_cer(output_dir, messages_template, dataset_p40)
    
    # Calculate metrics
    mean_cer = df_results['CER'].mean()
    median_cer = df_results['CER'].median()
    memory_usage = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    speed = 5000 / (time.time() - start_time)
    
    return {
        'params': params,
        'mean_cer': mean_cer,
        'median_cer': median_cer,
        'memory_usage': memory_usage,
        'speed': speed
    }


# Run function

In [13]:
# Define hyperparameter grid
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [4, 8]
accumulation_steps = [1, 2]
dataset_ranges = [(0, 55), (0, 100)]
image_factors = [28, 32]
eval_steps_list = [0]
max_steps_list = [50]

# Create combinations of hyperparameters
param_grid = list(itertools.product(learning_rates, batch_sizes, accumulation_steps, dataset_ranges, image_factors, eval_steps_list, max_steps_list))

# Load dataset
dataset_p40 = load_from_disk("data/processed/dataset_p40")

In [17]:
len(param_grid)

48

In [18]:

# Run experiments in parallel
results = []
start_time = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(train_and_evaluate, param_grid))

# Convert results to DataFrame
df_results = pd.DataFrame(results)


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.94s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.50it/s]
Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2-VL-2B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.








Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]
Some weights of Qwen2VLForConditionalGeneration were not initialized from the model checkpoint at Qwen/Qwen2-VL-2B-Instruct and are newly initialized: ['lm_head.weight'

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [ ]:

# Visualize results
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_results, x='mean_cer', y='median_cer', size='memory_usage', hue='speed', palette='viridis', sizes=(20, 200))
plt.title('Performance Metrics Across Hyperparameter Combinations')
plt.xlabel('Mean CER')
plt.ylabel('Median CER')
plt.legend(title='Speed (steps/sec)')
plt.show()